# cloning in github repo

In [ ]:
!git clone https://github.com/deepmind/kinetics-i3d.git

In [ ]:
%cd /content/kinetics-i3d

In [ ]:
!python evaluate_sample.py

# necessary imports

In [ ]:
!pip install -q imageio
!pip install -q opencv-python
!pip install -q git+https://github.com/tensorflow/docs

In [ ]:
from absl import logging
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
logging.set_verbosity(logging.ERROR)
import random
import re
import os
import tempfile
import ssl
import cv2
import numpy as np
import imageio
from IPython import display
from urllib import request 
from IPython.display import HTML
from base64 import b64encode

# Action detection

In [ ]:
# Utilities to fetch videos from UCF101 dataset
UCF_ROOT = "https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/"
_VIDEO_LIST = None
_CACHE_DIR = tempfile.mkdtemp()
unverified_context = ssl._create_unverified_context()

def list_ucf_videos():
  """Lists videos available in UCF101 dataset."""
  global _VIDEO_LIST
  if not _VIDEO_LIST:
    index = request.urlopen(UCF_ROOT, context=unverified_context).read().decode("utf-8")
    videos = re.findall("(v_[\w_]+\.avi)", index)
    _VIDEO_LIST = sorted(set(videos))
  return list(_VIDEO_LIST)

In [ ]:
def fetch_ucf_video(video):
  """Fetchs a video and cache into local filesystem."""
  cache_path = os.path.join(_CACHE_DIR, video)
  if not os.path.exists(cache_path):
    urlpath = request.urljoin(UCF_ROOT, video)
    print("Fetching %s => %s" % (urlpath, cache_path))
    data = request.urlopen(urlpath, context=unverified_context).read()
    open(cache_path, "wb").write(data)
  return cache_path

In [ ]:
def crop_center_square(frame):
  y, x = frame.shape[0:2]
  min_dim = min(y, x)
  start_x = (x // 2) - (min_dim // 2)
  start_y = (y // 2) - (min_dim // 2)
  return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

In [ ]:
def load_video(path, max_frames=0, resize=(224, 224)):
  cap = cv2.VideoCapture(path)
  frames = []
  try:
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      frame = crop_center_square(frame)
      frame = cv2.resize(frame, resize)
      frame = frame[:, :, [2, 1, 0]]
      frames.append(frame)
      
      if len(frames) == max_frames:
        break
  finally:
    cap.release()
  return np.array(frames) / 255.0

In [ ]:
def to_gif(images):
  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images, fps=25)
  return embed.embed_file('./animation.gif')

**downloading labels**

In [ ]:
#kinetics 600 labels
KINETICS_URL = "https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map_600.txt"
with request.urlopen(KINETICS_URL) as obj:
  labels = [line.decode("utf-8").strip() for line in obj.readlines()]
print("Found %d labels." % len(labels))

**Download videos from youtube or UCF 101**

In [ ]:
!pip install pytube3
from pytube import YouTube 

In [ ]:
video = YouTube('URL')
video.streams.filter(file_extension = "mp4").all()

In [ ]:
video.streams.get_by_itag(135).download('PATH')

In [ ]:
mp4 = open('PATH.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

*list ucf videos to download*

In [ ]:
len(list_ucf_videos())

In [ ]:
list_ucs_vide0s()['from':'to']

In [ ]:
list_ucf_videos()[0]

*view dataset according to video categories*

In [ ]:
# Get the list of videos in the dataset.
ucf_videos = list_ucf_videos()
  
categories = {}
for video in ucf_videos:
  category = video[2:-12]
  if category not in categories:
    categories[category] = []
  categories[category].append(video)
print("Found %d videos in %d categories." % (len(ucf_videos), len(categories)))

for category, sequences in categories.items():
  summary = ", ".join(sequences[:2])
  print("%-20s %4d videos (%s, ...)" % (category, len(sequences), summary))


In [ ]:
video_path = fetch_ucf_video("v_CricketShot_g04_c02.avi")
sample_video = load_video(video_path)

In [ ]:
sample_video.shape

In [ ]:
i3d = hub.load("https://tfhub.dev/deepmind/i3d-kinetics-600/1").signatures['default']

In [ ]:
def predict(sample_video):
  # Add a batch axis to the to the sample video.
  model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]

  logits = i3d(model_input)['default'][0]
  probabilities = tf.nn.softmax(logits)
  finalLabel = []
  print("Top 5 actions:")
  for i in np.argsort(probabilities)[::-1][:5]:
   print(f"  {labels[i]:22}: {probabilities[i] * 100:5.2f}%")
   #print(labels[:i])
   finalLabel.append(labels[i])
  print(finalLabel)
   

In [ ]:
predict(sample_video)

In [ ]:
!curl -O https://upload.wikimedia.org/wikipedia/commons/8/86/End_of_a_jam.ogv

In [ ]:
video_path = "End_of_a_jam.ogv"

In [ ]:
sample_video = load_video('/content/4.avi')[:100]
sample_video.shape

In [ ]:
to_gif(sample_video)

In [ ]:
predict(sample_video)

#server connection (Anvil code)

In [ ]:
!pip install anvil-uplink

In [ ]:
import anvil.server
anvil.server.connect("3FVQSCHIC4RFMRKBGFNPSQC7-SLFH33LGN7LS4UZ6")

In [ ]:
import anvil.media
@anvil.server.callable
def recog_actions(sample_video):
  # Add a batch axis to the to the sample video.
  with anvil.media.TempFile(sample_video) as filename:
    if os.path.exists('/content/frame0.jpg'):
      !rm /content/frame0.jpg
    cap= cv2.VideoCapture(filename)
    i=0
    for i in range(1):
      ret, frame = cap.read()
      if ret == False:
        break
    cv2.imwrite('frame'+str(i)+'.jpg',frame)
    i+=1
 
    cap.release()
    cv2.destroyAllWindows()

    sample_video = load_video(filename)[:100]
    model_input = tf.constant(sample_video, dtype=tf.float32)[tf.newaxis, ...]
    logits = i3d(model_input)['default'][0]
    probabilities = tf.nn.softmax(logits)
    prediction =[]
    #print("Top 5 actions:")
    for i in np.argsort(probabilities)[::-1][:5]:
      print(f"  {labels[i]:22}: {probabilities[i] * 100:5.2f}%")    
      prediction.append(labels[i])
    return(prediction[0],anvil.media.from_file('/content/frame0.jpg', 'jpg'))

# Trying Streamlit 

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

In [ ]:
!python -m http.server 8000

In [ ]:
from IPython.display import Javascript

def show_port(port, height=400):
  display(Javascript("""
  (async ()=>{
    fm = document.createElement('iframe')
    fm.src = await google.colab.kernel.proxyPort(%s)
    fm.width = '95%%'
    fm.height = '%d'
    fm.frameBorder = 0
    document.body.append(fm)
  })();
  """ % (port, height) ))